# Assignment parallel computing group 3

In [28]:
import fake_enformer 
fake_enformer.predict('hg38:chr4:8848348:A:T')
#  -0.0007830759576305497

-0.0007830759576305497

In [33]:
fake_enformer.predict('hg38:chr4:8848348:A:T')

-0.0007830759576305497

In [45]:
!cat /lustre1/project/stg_00079/teaching/data/snps.annotated.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.19+htslib-1.19.1
##bcftoolsCommand=mpileup -Ou -f /staging/leuven/stg_00079/teaching/hg38_9/chr9.fa 020.bwa/TLE66_N.bam 020.bwa/TLE66_T.bam
##reference=file:///staging/leuven/stg_00079/teaching/hg38_9/chr9.fa
##contig=<ID=chr9,length=138394717>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of raw reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of raw reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPBZ,Number=1,Type=Float,Description="Mann-Whitney U-z test of Read Po

## Import libraries

In [14]:
import vcfpy
import pandas as pd
from multiprocessing import Pool
import multiprocessing

## Configuration

In [35]:
genome = "hg38"
vcf_path = '/lustre1/project/stg_00079/teaching/data/snps.annotated.vcf'

In [79]:
reader = vcfpy.Reader.from_path(vcf_path)

In [37]:
def process_variant(variant_str):
    """Wrapper function for prediction"""
    return fake_enformer.predict(variant_str)

### Generate all variant strings

In [78]:
variant_strings = []
records = []

for i, record in enumerate(reader):
    print(type(record))
    assert len(record.ALT) == 1
    chrom = record.CHROM
    pos = record.POS
    ref = record.REF
    alt = record.ALT[0].value

    variant_str = f"{genome}:{chrom}:{pos}:{ref}:{alt}"
    variant_strings.append(variant_str)
    records.append(record)

    

In [ ]:
#variant_strings

## Multiprocessing

In [24]:
# Create process pool
num_processes = multiprocessing.cpu_count()
num_processes

72

In [41]:
with Pool(processes=num_processes,) as pool:
    results = pool.map(fake_enformer.predict, variant_strings, chunksize=100)

# Process results as needed
print(f"Processed {len(results)} variants")

Processed 3499 variants


Predict the first 10 snps without multiprocessing

In [32]:
results = map(fake_enformer.predict, variant_strings[:10])
print(list(results))

[]


## Save to new VCF (Still WIP)

In [85]:
# Update header with new FORMAT field
new_header = reader.header.copy()
new_header.add_format_line({
    'ID': 'EF',
    'Number': '1',
    'Type': 'Float',
    'Description': 'Enformer prediction score'
})

True

In [91]:
writer = vcfpy.Writer.from_path("snps_annotated_new.vcf", new_header)
for i, record in enumerate(reader):
    if results[i] is not None:
        # Add prediction to INFO field
        record.INFO['EF'] = [results[i]]
    writer.write_record(record)

print(f"Successfully wrote {len(results)} variants")

Successfully wrote 3499 variants


In [89]:
!cat snps_annotated_new.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.19+htslib-1.19.1
##bcftoolsCommand=mpileup -Ou -f /staging/leuven/stg_00079/teaching/hg38_9/chr9.fa 020.bwa/TLE66_N.bam 020.bwa/TLE66_T.bam
##reference=file:///staging/leuven/stg_00079/teaching/hg38_9/chr9.fa
##contig=<ID=chr9,length=138394717>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of raw reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of raw reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPBZ,Number=1,Type=Float,Description="Mann-Whitney U-z test of Read Po